# TD4 - Exercise 2

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("Dataset TD4.xlsx")

In [3]:
df

,transaction date (1=1day=24 hours),bid-ask spread,volume of the transaction (if known),Sign of the transaction,Price (before transaction)
0,0.000202,0.1100,8.0,-1,100.000
1,0.001070,0.1030,NaN,1,99.984
2,0.001496,0.1015,NaN,-1,100.029
3,0.003336,0.0920,NaN,1,99.979
4,0.003952,0.1106,NaN,1,100.060
...,...,...,...,...,...
996,0.981441,0.0834,79.0,1,101.070
997,0.981875,0.1010,NaN,-1,101.120
998,0.986784,0.1007,NaN,-1,100.998
999,0.991232,0.1153,3.0,-1,100.958


### Estimate all the parameters of the model of Almgren and Chriss. Is this model well specified?

In [4]:
df = df.rename(columns={
    "transaction date (1=1day=24 hours)": "t",
    "bid-ask spread": "spread",
    "volume of the transaction (if known)": "volume",
    "Sign of the transaction": "sign",
    "Price (before transaction)": "price"
})

df.sort_values(by = "t", inplace = True)

df["t_bin"] = (df["t"] // 0.01) * 0.01
df["t_bin"] = df["t_bin"].round(2)
df_grouped = df.groupby("t_bin").agg({
    "spread": "last", 
    "volume": "last",
    "sign": "last",
    "price": "last"
}).reset_index()
df_grouped

,t_bin,spread,volume,sign,price
0,0.00,0.0969,32.0,1,99.989
1,0.01,0.1062,8.0,1,100.022
2,0.02,0.1133,141.0,-1,99.890
3,0.03,0.1016,121.0,1,99.522
4,0.04,0.1102,52.0,-1,99.232
...,...,...,...,...,...
95,0.95,0.0884,213.0,-1,100.887
96,0.96,0.1105,150.0,-1,101.184
97,0.97,0.0930,17.0,1,101.000
98,0.98,0.1007,79.0,-1,100.998


In [5]:
df["volume"].max()

694.0

In [6]:
sigma = np.std((df_grouped["price"] - df_grouped["price"].shift(1)) / df_grouped["price"])
sigma = np.sqrt(100*252)*sigma
sigma

0.3792033106829055

In [7]:
T = 24
tau = 1/T
y = df_grouped["price"] - df_grouped["price"].shift(1) - (df_grouped["spread"]/2)*(df_grouped["sign"].shift(1) - df_grouped["sign"]) 
q = -1 * tau * df_grouped["volume"] / 0.01
a = (df_grouped["volume"].shift(1) - df_grouped["volume"]) / 0.01

temp = pd.concat([y, q, a], axis=1)
temp.columns = ["y", "q", "a"]
temp = temp.dropna()

y = temp["y"].to_numpy()
X = np.column_stack((temp["q"].to_numpy(), temp["a"].to_numpy()))

beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)

gamma_hat = beta_hat[0]
eta_hat   = beta_hat[1]

print("gamma =", gamma_hat)
print("eta   =", eta_hat)

gamma = 3.162431990232732e-05
eta   = 1.3838859299812787e-06


In [12]:
l = 2 * 10**(-6)
K = np.sqrt(l * sigma * sigma / eta_hat)
X_min = 1
X_moy = 400
X_big = 800
x_k_min = []
x_k_moy = []
x_k_big = []
for k in range(0, 25):
    x_k_min.append(np.sinh(K*(T - k)) * X_min / np.sinh(K*T))
    x_k_moy.append(np.sinh(K*(T - k)) * X_moy / np.sinh(K*T))
    x_k_big.append(np.sinh(K*(T - k)) * X_big / np.sinh(K*T))

for i in range(0, 25):
    print(x_k_min[i],";", x_k_moy[i],";", x_k_big[i])

1.0 ; 399.99999999999994 ; 799.9999999999999
0.6338989272765297 ; 253.55957091061188 ; 507.11914182122376
0.4018278497227113 ; 160.7311398890845 ; 321.462279778169
0.2547182422706917 ; 101.88729690827665 ; 203.7745938165533
0.16146561944529306 ; 64.58624777811723 ; 129.17249555623445
0.10235288117104274 ; 40.9411524684171 ; 81.8823049368342
0.06488137871319351 ; 25.952551485277404 ; 51.90510297055481
0.04112823181857453 ; 16.45129272742981 ; 32.90258545485962
0.026071134837880133 ; 10.428453935152053 ; 20.856907870304106
0.016526453048360364 ; 6.610581219344145 ; 13.22116243868829
0.010476082933668463 ; 4.190433173467385 ; 8.38086634693477
0.006640749451123976 ; 2.6562997804495905 ; 5.312599560899181
0.004209519333576593 ; 1.683807733430637 ; 3.367615466861274
0.002668339398587265 ; 1.067335759434906 ; 2.134671518869812
0.0016913464361807626 ; 0.676538574472305 ; 1.35307714894461
0.0010719675111500115 ; 0.42878700446000456 ; 0.8575740089200091
0.0006792427010879456 ; 0.2716970804351782